<a href="https://colab.research.google.com/github/aatish-garje/Schedule-vs-Dispatch/blob/main/Schedule_vs_Dispatch_Master_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

!pip install openpyxl

In [107]:
dispatch_df=pd.read_excel("/content/drive/MyDrive/Dispatch till 14.07.2025.XLSX")
schedule_power=pd.read_excel("/content/drive/MyDrive/REV 1 CUSTOMER SCHEDULE -July - 2025.xlsx", sheet_name='POWER', header=3)
schedule_mech=pd.read_excel("/content/drive/MyDrive/REV 1 CUSTOMER SCHEDULE -July - 2025.xlsx", sheet_name='MECH', header=3)

In [108]:
dispatch_df['Sold-to Party'] = dispatch_df.apply(
    lambda x: str(x['Sold-to Party']) + '.'
    if (x['Plant'] == 2000 and not str(x['Sold-to Party']).upper().startswith('V'))
    else x['Sold-to Party'],
    axis=1
)

In [109]:
dispatch_df = dispatch_df[~dispatch_df['Material'].astype(str).str.startswith('C')]

In [110]:
dispatch_df.loc[dispatch_df['Inv Qty'] == 0, 'Inv Qty'] = dispatch_df['Kit Qty']

In [111]:
dispatch_df = dispatch_df[dispatch_df['Material'] != 8043975905]

In [112]:
dispatch_df = dispatch_df[dispatch_df['Customer Group'] == 10]

In [113]:
# 1️⃣ Separate Sales Order No starting with '10' (keep untouched)
keep_sales_order_10 = dispatch_df[dispatch_df['Sales Order No'].astype(str).str.startswith('10')]

# 2️⃣ Remaining data where Sales Order No does NOT start with '10'
remaining = dispatch_df[~dispatch_df['Sales Order No'].astype(str).str.startswith('10')]

# 3️⃣ Identify Billing Doc No. with duplicates (count > 1)
billing_doc_counts = remaining['Billing Doc No.'].value_counts()
duplicates = billing_doc_counts[billing_doc_counts > 1].index.tolist()

# 4️⃣ Split duplicates and unique separately
duplicates_df = remaining[remaining['Billing Doc No.'].isin(duplicates)]
unique_df = remaining[~remaining['Billing Doc No.'].isin(duplicates)]

# 5️⃣ In duplicates, keep only Item == 10
duplicates_df = duplicates_df[duplicates_df['Item'] == 10]

# 6️⃣ Combine: keep_sales_order_10 + unique + cleaned duplicates
dispatch_df = pd.concat([keep_sales_order_10, unique_df, duplicates_df], ignore_index=True)

In [114]:
dispatch_summary = dispatch_df.groupby(['Sold-to Party', 'Material'], as_index=False)['Inv Qty'].sum()
dispatch_summary.rename(columns={'Inv Qty': 'Dispatch Qty'}, inplace=True)

In [115]:
schedule_power['Code'] = schedule_power['Code'].astype(str)
schedule_power['Part Number'] = schedule_power['Part Number'].astype(str)

schedule_mech['Code'] = schedule_mech['Code'].astype(str)
schedule_mech['Part Number'] = schedule_mech['Part Number'].astype(str)

In [116]:
# Power Schedule Merge
schedule_power = pd.merge(
    schedule_power,
    dispatch_summary,
    left_on=['Code', 'Part Number'],
    right_on=['Sold-to Party', 'Material'],
    how='left'
)

# Mech Schedule Merge
schedule_mech = pd.merge(
    schedule_mech,
    dispatch_summary,
    left_on=['Code', 'Part Number'],
    right_on=['Sold-to Party', 'Material'],
    how='left'
)

# Fill NaN with 0
schedule_power['Dispatch Qty'] = schedule_power['Dispatch Qty'].fillna(0)
schedule_mech['Dispatch Qty'] = schedule_mech['Dispatch Qty'].fillna(0)

columns_to_keep_power = [
    'Code', 'Customer', 'MODEL', 'BILLING PLANT', 'Part Number',
    'Customer Part', 'Description', 'Initial Schedule',
    'REV-1', 'REV-2'
]
# Columns starting with Marketing Requirement
marketing_columns_power = [col for col in schedule_power.columns if col.startswith('Marketing Requirement')]

# Rearrange Dispatch Qty at the end
final_columns_power = columns_to_keep_power + marketing_columns_power + ['Dispatch Qty']
schedule_power = schedule_power[final_columns_power]

columns_to_keep_power = [
    'Code', 'Customer', 'MODEL', 'BILLING PLANT', 'Part Number',
    'Customer Part', 'Description', 'Initial Schedule',
    'REV-1', 'REV-2'
]
# Columns starting with Marketing Requirement
marketing_columns_power = [col for col in schedule_power.columns if col.startswith('Marketing Requirement')]

# Rearrange Dispatch Qty at the end
final_columns_power = columns_to_keep_power + marketing_columns_power + ['Dispatch Qty']
schedule_power = schedule_power[final_columns_power]

In [117]:
with pd.ExcelWriter('/content/Schedule_with_Dispatch.xlsx') as writer:
    schedule_power.to_excel(writer, sheet_name='Power', index=False)
    schedule_mech.to_excel(writer, sheet_name='Mech', index=False)